## Load Libraries

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import re
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle
import pickle
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

import matplotlib.font_manager
from matplotlib import style
style.use('seaborn') or plt.style.use('seaborn')

from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
import plotly.graph_objects as go
import seaborn as sns

from collections import Counter
import operator


from xgboost import plot_importance


from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo

from sklearn.decomposition import FactorAnalysis

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import scipy.stats as ss


from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import Lasso, MultiTaskLasso, Ridge, ElasticNet
import math

#from regressors import stats


import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import scipy.stats as ss

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle


import scipy
from sklearn import metrics


from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import colorsys
import matplotlib.colors as cconv
from statannot import add_stat_annotation

In [ ]:
import my_bib as mb

## Define functions

In [ ]:
def get_color(var, X_cat_, color_dict_):

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.newname)

    index_code = X_name.index(var)
    cat = X_factors[index_code]

    return color_dict_[cat]

color_dict = {}
color_dict['Country'] = [40/255, 43/255, 95/255]
color_dict['HF'] = '#A31300'
color_dict['LSF'] = '#FF9E0D'
color_dict['PSF'] = '#FF4800'
color_dict['SDHF'] = '#1C4F9E'
color_dict['DF'] = '#009E32'

def get_bar_colors(data_, X_cat_):
    color_dict = {}
    color_dict['Country'] = [40/255, 43/255, 95/255]
    color_dict['HF'] = '#A31300'
    color_dict['LSF'] = '#FF9E0D'
    color_dict['PSF'] = '#FF4800'
    color_dict['SDHF'] = '#1C4F9E'
    color_dict['DF'] = '#009E32'
    
    #color_dict = {}
    #color_dict = {'DF':'#5975a4', 'MF':'#cc8963', 'SF':'#5f9e6e', 'SLF':'#b55d60',
    #              'n':'#857aab', 'country':'#8d7866'}#, '#d095bf'}

    X_factors = list(X_cat_.factors)
    X_name = list(X_cat_.newname)

    bar_color_total = []
    for i in list(data_.Features):
        if(i == 'Country'):
            bar_color_total.append(color_dict['Country'])
            continue
        index_code = X_name.index(i)
        cat = X_factors[index_code]
        bar_color_total.append(color_dict[cat])
    return bar_color_total


def plot_estimate_value(regression_model, X_cat_ = [], title = '',  xlim =[0, 2] ,fig_size = (8,12), size = 14, pvalue_type = 'False'):

    df = regression_model[0]
    df.index.name = 'Features'
    df = df.iloc[1:-3, 0:-1]

    for i in range(3):
            #print(i, X_RAW_edu_level[X_RAW_edu_level.columns[i]].dtype)
        df[df.columns[i]] = np.abs(pd.to_numeric(df[df.columns[i]],errors = 'coerce'))

    df = df.reset_index()
    data = df.sort_values('Estimate', ascending=False)

    
    if(len(X_cat_)>0):
        
        bar_color = get_bar_colors(data, X_cat_)
        
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, palette =bar_color)
        plt.xlim(xlim)
    else:
        plt.title(title)
        sns.barplot(x="Estimate", y="Features", data = data, color = 'darkblue')
        plt.xlim(xlim)


    y_step=0  
    for i in range(df.shape[0]):
        if(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.01):
            color = 'green'
        elif(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.05):
            color = 'gray'
        else:
            color = 'red'        
        
        if(pvalue_type == 'color'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]-0.005, y_step, 
                                 '' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],2)),
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'white',
                                 bbox=dict(boxstyle="round",
                                           ec=color,
                                            fc=color,
                                           )
                                 )
                y_step+=1

        elif(pvalue_type == 'value'):
                plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                 '(' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],8))+')',
                                 size= size, rotation=0.,
                                 ha="left", va="center", color = 'black',

                                 )
                y_step+=1
        else:
            if(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.01):
                
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '**',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )
            elif(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]>= 0.01 and df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step]<0.05):
                  
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '*',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )  
            else:
                    plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]+0.005, y_step, 
                                     '',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )      
            y_step+=1
            
         
    text_diff =xlim[1]/2.2
    plt.text(xlim[1] - text_diff, df.shape[0]-1.5,r'$ R^2 $(' + str(np.round(regression_model[1],2)) + ') \t$F^2 $(' + str(np.round(regression_model[3],2)) + ')',
                             size= 12, rotation=0.,
                             ha="left", va="center", color = 'black',
                             bbox=dict(boxstyle="round",
                                       ec='gray',
                                        fc='gray',
                                       )
                             )


    plt.locator_params(axis='x', nbins=4)




def plot_estimate_value_no_sort(regression_model, title = '',  xlim =[0, 2] ,fig_size = (8,12), size = 14, ylabel = True, ylabelR = False ):

    df = regression_model
    df.index.name = 'Features'
    df = df.iloc[1:-3, 0:-1]

    for i in range(3):
            #print(i, X_RAW_edu_level[X_RAW_edu_level.columns[i]].dtype)
        df[df.columns[i]] = np.abs(pd.to_numeric(df[df.columns[i]],errors = 'coerce'))

    df = df.reset_index()

    #plt.figure(figsize=(8,12))
    plt.title(title)
    sns.barplot(x="Estimate", y="Features", data = df, color = 'Brown')
    if(ylabel == False):
        plt.ylabel('')
        plt.yticks([])
    plt.xlim(xlim)
    
    if(ylabelR):
        plt.tick_params (axis = 'y', which = 'both', labelleft = False, labelright = True)

    y_step=0  
    for i in range(df.shape[0]):
        if(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.01):
            color = 'green'
        elif(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)<=0.05):
            color = 'gray'
        else:
            color = 'red'        
        
        plt.text(df.sort_values('Estimate', ascending=False)['Estimate'].iloc[y_step]-0.005, y_step, 
                         '' + str(np.round(df.sort_values('Estimate', ascending=False)['p value'].iloc[y_step],3)),
                         size= size, rotation=0.,
                         ha="left", va="center", color = 'white',
                         bbox=dict(boxstyle="round",
                                   ec=color,
                                    fc=color,
                                   )
                         )
        y_step+=1
        

        
def conf_interval(database):
    r_squared_list = []

    y_database = database['Barthel']
    X_database = database.drop('Barthel', axis=1)

    for i in range(0,100,10):

        X_train, X_test, y_train, y_test = train_test_split(X_database, y_database, train_size=0.75, test_size=.25, random_state=i)

        opt_Ridge = BayesSearchCV(
            Ridge(),
            {
                'alpha': ( 0.0001, 0.01, 0.001),
                'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                'max_iter': (1000, 10000, 100000, 1000000),
            },
            n_iter=10,
            random_state=i, 
            scoring='r2',
            cv=3
        )

        opt_Ridge.fit(X_train, y_train)

        r_squared_list.append(opt_Ridge.score(X_test, y_test))


    print(r'99% confidence interval (+-', np.round(np.std(r_squared_list)*2.58, 4),')')
    return np.round(np.std(r_squared_list)*2.58, 4)


def adj_r2_score_and_r2_score(clf, X, y):
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = r2_score(y, clf.predict(X))
    return [1 - (1 - r_squared) * ((n - 1) / (n - p - 1)), r_squared]


def mse(clf, X, y):
    return mean_squared_error(y, clf.predict(X))

def rmse(clf, X, y):
    mse = mean_squared_error(y, clf.predict(X))
    return math.sqrt(mse)    

def coef_se(clf, X, y):
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)

def coef_tval(clf, X, y):
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_tval_XGB_tree(clf, X, y):
    a = np.nan
    b = np.array(clf.feature_importances_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)

def coef_pval(clf, X, y):

    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def coef_pval_XGB_tree(clf, X, y):

    n = X.shape[0]
    t = coef_tval_XGB_tree(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def residuals(clf, X, y, r_type='standardized'):

    # Make sure value of parameter 'r_type' is one we recognize
    assert r_type in ('raw', 'standardized', 'studentized'), (
        "Invalid option for 'r_type': {0}".format(r_type))
    y_true = y.view(dtype='float')
    # Use classifier to make predictions
    y_pred = clf.predict(X)
    # Make sure dimensions agree (Numpy still allows subtraction if they don't)
    assert y_true.shape == y_pred.shape, (
        "Dimensions of y_true {0} do not match y_pred {1}".format(y_true.shape,
                                                                  y_pred.shape))
    # Get raw residuals, or standardized or standardized residuals
    resids = y_pred - y_true
    if r_type == 'standardized':
        resids = resids / np.std(resids)
    elif r_type == 'studentized':
        # Prepare a blank array to hold studentized residuals
        studentized_resids = np.zeros(y_true.shape[0], dtype='float')
        # Calcluate hat matrix of X values so you can get leverage scores
        hat_matrix = np.dot(
            np.dot(X, np.linalg.inv(np.dot(np.transpose(X), X))),
            np.transpose(X))
        # For each point, calculate studentized residuals w/ leave-one-out MSE
        for i in range(y_true.shape[0]):
            # Make a mask so you can calculate leave-one-out MSE
            mask = np.ones(y_true.shape[0], dtype='bool')
            mask[i] = 0
            loo_mse = np.average(resids[mask] ** 2, axis=0)  # Leave-one-out MSE
            # Calculate studentized residuals
            studentized_resids[i] = resids[i] / np.sqrt(
                loo_mse * (1 - hat_matrix[i, i]))
        resids = studentized_resids
    return resids


def f_squared(clf, X, y):

    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return r_squared  / (1 - r_squared)

def f_stat(clf, X, y):
    """Calculate summary F-statistic for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic value.
    """
    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return (r_squared / p) / ((1 - r_squared) / (n - p - 1))

def f_stat_pvalue(clf, X, y):
    """Calculate summary F-statistic p value for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic p value.
    """
    n = X.shape[0] # Esto se extrae par los grados de libertad del numeador y el denomindor (no. predictores, no. sujetos - no. predictores-1)
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    
    return np.round(scipy.stats.f.sf(f_stat(clf, X, y), n, (n - p - 1)), 15)

def compute_f_statistics(clf, X, y):
    return [f_stat(clf, X, y), f_stat_pvalue(clf, X, y)]



def summary(clf, X, y, xlabels=None, regressor = ''):

    print('Resumen del regresor ' + regressor + '\n')
    
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')

    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate','t value', 'p value']
    )
    
    if(regressor == 'XGBRegressor'):
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_[0]]), 6), np.round((clf.coef_), 6)))
        #coef_df['MSE'] = np.round(mse(clf, X, y), 6)
        #coef_df['RMSE'] = np.round(rmse(clf, X, y), 6)
        coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
        coef_df['p value'] = np.round(coef_pval(clf, X, y), 20)
        # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
            'Min': pd.Series(np.round(resids.min(), 4)),
            '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
            'Median': pd.Series(np.round(np.median(resids), 4)),
            '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
            'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
        # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)
        
        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
           r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
            f_sq))
    elif(regressor == 'XGBRegressorNoLinear'):
        coef_df = pd.DataFrame(
            index=['_intercept'] + list(xlabels),
            columns=['Estimate','t value', 'p value']
        )

        coef_df['Estimate'] = np.concatenate(
                (np.round(np.array([np.nan]), 6), np.round((clf.feature_importances_), 6)))

        coef_df['t value'] = np.round(coef_tval_XGB_tree(clf, X, y), 4)
        coef_df['p value'] = np.round(coef_pval_XGB_tree(clf, X, y), 20)
            # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
                'Min': pd.Series(np.round(resids.min(), 4)),
                '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
                'Median': pd.Series(np.round(np.median(resids), 4)),
                '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
                'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
            # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)

        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
               r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
                f_sq))
    else:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
        #coef_df['MSE'] = np.round(mse(clf, X, y), 6)
        #coef_df['RMSE'] = np.round(rmse(clf, X, y), 6)
        coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
        coef_df['p value'] = np.round(coef_pval(clf, X, y), 20)
        # Create data frame to summarize residuals
        resids = residuals(clf, X, y, r_type='raw')
        resids_df = pd.DataFrame({
            'Min': pd.Series(np.round(resids.min(), 4)),
            '1Q': pd.Series(np.round(np.percentile(resids, q=25), 4)),
            'Median': pd.Series(np.round(np.median(resids), 4)),
            '3Q': pd.Series(np.round(np.percentile(resids, q=75), 4)),
            'Max': pd.Series(np.round(resids.max(), 4)),
        }, columns=['Min', '1Q', 'Median', '3Q', 'Max'])
        # Output results
        print("Residuals:")
        print(resids_df.to_string(index=False))
        print('\n')
        print('Coefficients:')
        print(coef_df.to_string(index=True))
        print('---')
        
        r_sq = adj_r2_score_and_r2_score(clf, X, y)[1]
        r_sq_adj = adj_r2_score_and_r2_score(clf, X, y)[0]
        f_sq = f_squared(clf, X, y)
        
        mse_ = np.round(mse(clf, X, y), 6)
        rmse_ = np.round(rmse(clf, X, y), 6)
        
        print('R-squared:  {0:.5f},    Adjusted R-squared:  {1:.5f}'.format(
           r_sq, r_sq_adj))
        print('F-squared:  {0:.5f}'.format(
            f_sq))
        print('MSE:  {0:.5f}'.format(
            mse_))        
        print('RMSE:  {0:.5f}'.format(
            rmse_))        
    print('---------------------------------------------------------------------------\n\n')
    
    
    
    empty_str = []
    for i in range(coef_df.shape[0]):
        empty_str.append('')
    
    coef_df['value'] = empty_str
    
    coef_df = coef_df.T
    coef_df['R-squared'] = ['','','', r_sq]
    coef_df['Adjusted R-squared'] = ['','','', r_sq_adj]
    coef_df['F-squared'] = ['','','', f_sq]
    #coef_df['MSE'] = ['','','', mse_]
    #coef_df['RMSE'] = ['','','', rmse_]
    return [coef_df.T, r_sq, r_sq_adj, f_sq, mse_, rmse_]



def summary_validation(X_, y_, opt_Ridge_ = ''):
    
    from statsmodels.stats.multitest import fdrcorrection
    #from statsmodels.stats.multitest import fisher_combined_pvalues
    from scipy.stats import combine_pvalues



    lista_vars = list(X_)
    
    coef_array = np.zeros([len(lista_vars)+1, 10])
    coef_t_value = np.zeros([len(lista_vars)+1, 10])
    coef_p_value = np.zeros([len(lista_vars)+1, 10])

    r2_list =[]
    f2_list =[]
    f_list =[]
    f_p_value_list =[]
    mse_list =[]
    rmse_list =[]
    for iter_ in tqdm(range(0,10), total=len(range(0, 10)),leave=True, mininterval=1, ascii=True,
                            colour='green', desc='Training model'):
        # Stratified split
        

        X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.30, random_state=iter_)

        model = Ridge(alpha=0.1, solver = opt_Ridge_.best_params_['solver'], 
                         max_iter=opt_Ridge_.best_params_['max_iter'])

        model.fit(X_train, y_train)

        coef_array[0,iter_] = np.abs(model.intercept_)
        coef_array[1::,iter_] = np.abs(model.coef_)

        #print(iter_, coef_se(model, X_test, y_test), np.abs(coef_se(model, X_test, y_test)))

        coef_t_value[:,iter_] = np.abs(np.round(coef_tval(model, X_test, y_test), 30))
        coef_p_value[:,iter_] = np.round(coef_pval(model, X_test, y_test), 30)

        #print(coef_t_value[:,iter_], coef_p_value[:,iter_])
        #coef_df['t value'] = np.abs(np.round(coef_tval(clf, X, y), 4))
        #coef_df['p value'] = np.round(coef_pval(clf, X, y), 20)

        f2_list.append(f_squared(model, X_test, y_test))
        f_list.append(f_stat(model, X_test, y_test))
        f_p_value_list.append(f_stat_pvalue(model, X_test, y_test))
        r2_list.append(r2_score(y_test, model.predict(X_test)))
        
        mse_list.append(np.round(mse(model, X_test, y_test), 6))
        rmse_list.append(np.round(rmse(model, X_test, y_test), 6))


    r2_list_array = np.array(r2_list)
    # finding the 1st quartile
    q1 = np.quantile(r2_list_array, 0.25)
 
    # finding the 3rd quartile
    q3 = np.quantile(r2_list_array, 0.75)
    med = np.median(r2_list_array)

    # finding the iqr region
    iqr = q3-q1

    # finding upper and lower whiskers
    upper_bound = q3+(1.5*iqr)
    lower_bound = q1-(1.5*iqr)
    #print(iqr, upper_bound, lower_bound)

    index_del = []     

    for i in range(r2_list_array.shape[0]):
        if((r2_list_array[i]<= lower_bound) | (r2_list_array[i] >= upper_bound)):
            index_del.append(i)

    
    
    r2_list = list(np.delete(r2_list_array,index_del, axis=0 ))

    coef_df = pd.DataFrame(
            index=['_intercept'] + lista_vars,
            columns=['Estimate mean','t value mean', 'p value mean', 'p value fdr', 
                     'p value stouffer', 'p value fisher', 
                      'p value tippett', 'Estimate std']
        )
    
    df_coef_value = {}
    df_p_value = {}
    df_t_value = {}
    lista_vars_interc = ['_intercept'] + lista_vars
    for i in range(len(lista_vars_interc)):
        df_coef_value[lista_vars_interc[i]] = coef_array[i,:]
        df_p_value[lista_vars_interc[i]] = coef_p_value[i,:]
        df_t_value[lista_vars_interc[i]] = coef_t_value[i,:]

    coef_array  = np.delete(coef_array,index_del, axis=1 )
    coef_t_value = np.delete(coef_t_value,index_del, axis=1 )
    coef_p_value = np.delete(coef_p_value,index_del, axis=1 )
    
    
    coef_array_mean = np.zeros([len(lista_vars)+1, 1])
    coef_t_value_mean = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_mean = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_fdr = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_stouffer = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_fisher = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_pearson = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_tippett = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_mudholkar_george = np.zeros([len(lista_vars)+1, 1])

    coef_array_std = np.zeros([len(lista_vars)+1, 1])

    print('shape',coef_array.shape, coef_t_value.shape, coef_p_value.shape, len(index_del))
    
    
    
    #coef_p_value = coef_p_value + 1e-9
    coef_p_value = np.where(coef_p_value == 0, 1e-30, coef_p_value)
    
    f_p_value_list_a = np.array(f_p_value_list)
    f_p_value_list_a = np.where(f_p_value_list_a == 0, 1e-30, f_p_value_list_a)
    f_p_value_list = list(f_p_value_list_a)
    
    for i in range(len(lista_vars)+1):
        coef_array_mean[i] = coef_array[i,:].mean()
        coef_t_value_mean[i]= coef_t_value[i,:].mean()
        coef_p_value_mean[i]= coef_p_value[i,:].mean()
        coef_p_value_fdr[i]= fdrcorrection(coef_p_value[i,:], alpha=0.05, is_sorted=False )[1].max()
        coef_p_value_stouffer[i] = combine_pvalues(coef_p_value[i,:], method='stouffer')[1]
        coef_p_value_fisher[i] = combine_pvalues(coef_p_value[i,:], method='fisher')[1]
        #coef_p_value_pearson[i] = combine_pvalues(coef_p_value[i,:], method='pearson')[1]
        coef_p_value_tippett[i] = combine_pvalues(coef_p_value[i,:], method='tippett')[1]
        #coef_p_value_mudholkar_george[i] = combine_pvalues(coef_p_value[i,:], method='mudholkar_george')[1]
        
        coef_array_std[i] = coef_array[i,:].std()
        # Se corrige por FDR y se reporta el valor máximo

    coef_df['Estimate mean'] = coef_array_mean
    coef_df['t value mean'] = coef_t_value_mean
    coef_df['p value mean'] = coef_p_value_mean
    coef_df['p value fdr'] = coef_p_value_fdr
    coef_df['p value stouffer'] = coef_p_value_stouffer
    coef_df['p value fisher'] = coef_p_value_fisher
    #coef_df['p value pearson'] = coef_p_value_pearson
    coef_df['p value tippett'] = coef_p_value_tippett
    #coef_df['p value mudholkar_george'] = coef_p_value_mudholkar_george
    coef_df['Estimate std'] = coef_array_std


    empty_str = []
    for i in range(coef_df.shape[0]):
        empty_str.append('')

    coef_df['value'] = empty_str

    coef_df = coef_df.T
    coef_df['R-squared'] = ['','','','','','','', '', np.mean(r2_list)]
    coef_df['CI'] = ['','','','','','','', '', np.std(r2_list)*1.96]
    coef_df['F-squared'] = ['','','','','','','', '', np.mean(f2_list)]
    coef_df['F'] = ['','','','','','','', '', np.mean(f_list)]
    coef_df['F-pvalue fdr'] = ['','','','','','','', '', fdrcorrection(f_p_value_list, alpha=0.05, is_sorted=False )[1].max()]
    coef_df['F-pvalue stouffer'] = ['','','','','','','', '', combine_pvalues(f_p_value_list, method='stouffer')[1]]  
    coef_df['F-pvalue fisher'] = ['','','','','','','', '', combine_pvalues(f_p_value_list, method='fisher')[1]]  
    
    #coef_df['F-pvalue pearson'] = ['','','','','','','','','', '', combine_pvalues(f_p_value_list, method='pearson')[1]]  
    coef_df['F-pvalue tippett'] = ['','','','','','','', '',combine_pvalues(f_p_value_list, method='tippett')[1]]  
    #coef_df['F-pvalue mudholkar_george'] = ['','','','','','','','','', '', combine_pvalues(f_p_value_list, method='mudholkar_george')[1]]  
    #coef_df['F-pvalue_fdr'] = ['','','', fdrcorrection(f_p_value_list, alpha=0.05, method='indep', is_sorted=False )]
    coef_df = coef_df.T

    print('Coefficients:')
    display(coef_df)
    print(10*'---')
        
    #print(np.mean(r2_list),  np.std(r2_list)*1.96)
    
    return [coef_df, df_coef_value, df_p_value, df_t_value, mse_list, rmse_list, r2_list, f2_list]


In [ ]:
import math

def sigmoid(x):
    sig = 1 / (1 + math.exp(-x))
    return sig

def get_order_var(model_):

    df = model_[0]['Estimate'].iloc[1:-3]
    df = np.abs(pd.to_numeric(df,errors = 'coerce'))
    df = list(df.sort_values(ascending=False).index)
    return df



import matplotlib as mpl

def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

def plot_longitudinal(var, y_countrys_, models_, market_size_type = 'log',xlim = [0.25, 5.75], 
                      years = [2006, 2014, 2016],ylim = [10.0, 11.12], size_init = 1, size_mult = 40, 
                      size_edge_market = 2, xlabel = True, ylabel = True, ylabel_text = 'MMSE',color_dict_ = '', X_cat_ = ''):
    
    #X_cat = pd.read_csv('Data/var_name_color_code.csv', encoding='latin-1', sep=";")
    
    color = get_color(var, X_cat_, color_dict_)
    edge_color = 'black'

    y = []
    for i in range(len(y_countrys_)):
        y.append(y_countrys_[i].mean())
        #y = [y_countrys_[0].mean(), y_countrys_[1].mean(),
        #     y_countrys_[2].mean(), y_countrys_[3].mean(),
        #     y_countrys_[4].mean()]


    #var_ind = list(X_cat.oldname).index(var)
    #var_ = list(X_cat.newname)[var_ind]
    

     
    plt.plot(range(1, len(y_countrys_)+1), y, 'k--')

    for i in range(len(models_)):
        df = models_[i][0]
        y = np.abs(df.iloc[df.index.get_loc(var), df.columns.get_loc('Estimate')])
        
        if(market_size_type=='log'):
            if(y < 0.003):
                markersize_ = 0
            else:
                markersize_ = size_mult*(5 + np.log(y))
        elif(market_size_type=='log1'):
            markersize_ = np.log(100000*y)
        elif(market_size_type=='sigmoide'):
            markersize_ = size_mult*sigmoid(y)
        else:
            markersize_ = np.abs(size_mult*y) 
        
        markersize_ +=size_init
        plt.plot(i + 1, y_countrys_[i].mean(), marker="o",  
                 markeredgewidth = size_edge_market, markeredgecolor = edge_color, markerfacecolor = color, markersize= markersize_)
        
        
        p = df['p value'][var]
        
        p_text = ''
        if(p<= 0.01):
            p_text = '**'
        elif((p<= 0.05)):
            p_text = '*'
        
        
        plt.text(i + 1 , y_countrys_[i].mean(), p_text,
                                             size= 22, rotation=0.,
                                             ha="center", va="top", color = 'black',
                                            # bbox=dict(boxstyle="round", pad=0.1,
                                            #           ec='gray',
                                            #           fc='gray',
                                            #          )
                                             );
        
        #plt.text(i + 1 , ylim[0] - 0.25, str(round(markersize_,2)),
         #                                    size= 8, rotation=0.,
          #                                   ha="center", va="top", color = 'black',
                                            # bbox=dict(boxstyle="round", pad=0.1,
                                            #           ec='gray',
                                            #           fc='gray',
                                            #          )
           #                                  );
       # print('')
        r2 = str(np.round(models_[i][1],2))
        f2 = str(np.round(models_[i][3],2))
        #plt.text(i+1,10.15,r'$R^2$ ('+r2+') \n$F^2$ ('+f2+')',
        #                         size= 12, rotation=0.,
        #                         ha="center", va="top", color = 'black',
        #                         bbox=dict(boxstyle="round", pad=0.1,
        #                                   ec='ghostwhite',
        #                                    fc='ghostwhite',
        #                                  )
        #                         );

    plt.text(xlim[1] , ylim[1], var,
                                         size= 13, rotation=0.,
                                         ha="right", va="center", color = 'black',
                                        # bbox=dict(boxstyle="round", pad=0.1,
                                        #           ec='gray',
                                        #           fc='gray',
                                        #          )
                                         );

    plt.xlim(xlim)
    plt.ylim([ylim[0]-0.25, ylim[1]+0.25])
    

    if(xlabel):
        plt.xticks(list(range(1, len(years)+1)), years)
    else:
        plt.xticks(list(range(1, len(years)+1)), ['', '', '', '', ''])
    
    if(ylabel):
        plt.yticks(np.round(np.linspace(ylim[0], ylim[1], 4),2), np.round(np.linspace(ylim[0], ylim[1], 4),1))
    else:
        plt.yticks(np.round(np.linspace(ylim[0], ylim[1], 4),2), ['', '', '', ''])

    if(ylabel):
        plt.ylabel(ylabel_text, fontsize=13)
    else:
        plt.ylabel('')
        
    if(xlabel):
        plt.xlabel('Years', fontsize=13)
    else:
        plt.xlabel('')
        
def plot_r2_f2(models_, xlim = [0.25, 5.75], ylim = [10.0, 11.12]):
    
    X_cat = pd.read_csv('Data/var_name_color_code.csv', encoding='latin-1', sep=";")
    
    
    plt.plot(0.0)

    for i in range(len(models)):

        r2 = str(np.round(models_[i][1],2))
        f2 = str(np.round(models_[i][3],2))
        plt.text(i+1,1,r'$R^2$ ('+r2+') \n$F^2$ ('+f2+')',
                                 size= 12, rotation=0.,
                                 ha="center", va="top", color = 'black',
                                 #bbox=dict(boxstyle="round", pad=0.1,
                                 #          ec='ghostwhite',
                                 #           fc='ghostwhite',
                                 #         )
                                 );
        
    
    plt.xlim(xlim)
    plt.ylim([ylim[0], ylim[1]])
    plt.xticks([])
    plt.yticks([])

## Longitudinal

In [ ]:
sabe_chile = pd.read_excel('../Data/cross/SABE_chile.xlsx') # to use it for columns order
sabe_chile = sabe_chile.iloc[:,1::]

sabe_costarica = pd.read_excel('../Data/long/SABE_costarica_long.xlsx')
sabe_costarica = sabe_costarica.iloc[:,1::]

sabe_china_2011_2014 = pd.read_excel('../Data/long/SABE_china_2011_2014.xlsx')
sabe_china_2011_2014 = sabe_china_2011_2014.iloc[:,1::]

In [ ]:
sabe_china_2011_2014['edu_level'] = 0
sabe_china_2011_2014.loc[(sabe_china_2011_2014['Education']==88) | (sabe_china_2011_2014['Education']==99), 'edu_level'] = 1
sabe_china_2011_2014.loc[(sabe_china_2011_2014['Education']<=6), 'edu_level'] = 1
sabe_china_2011_2014.loc[(sabe_china_2011_2014['Education']>=7) & (sabe_china_2011_2014['Education']<=12), 'edu_level'] = 2
sabe_china_2011_2014.loc[(sabe_china_2011_2014['Education']>12), 'edu_level'] = 3

sabe_china_2011_2014 = sabe_china_2011_2014.drop('Education', axis=1)

sabe_china_2011_2014 = sabe_china_2011_2014.rename(columns={'edu_level': 'Education'})

In [ ]:
order_col = list(sabe_chile.columns)
del sabe_chile
order_col

order_col_long = ['Age', 'Sex', 'Diabetes', 'Education', 'Hypertension', 'Heart Disease', 'Mental Problems', 'Physical activity', 'Alcohol consumption',
       'Social activity', 'Smoking status', 'Income', 'Barthel_diff']
order_col_2011 = ['Age', 'Sex', 'Diabetes', 'Education', 'Hypertension', 'Heart Disease', 'Mental Problems', 'Physical activity', 'Alcohol consumption',
       'Social activity', 'Smoking status', 'Income', 'Barthel_2011', 'Barthel_diff']
order_col_2014 = ['Age', 'Sex', 'Diabetes', 'Education', 'Hypertension', 'Heart Disease', 'Mental Problems', 'Physical activity', 'Alcohol consumption',
       'Social activity', 'Smoking status', 'Income', 'Barthel_2014', 'Barthel_diff']

In [ ]:
X_costarica_01 = sabe_costarica.copy()

In [ ]:
list_drop = ['FD_none_Edad_a01b', 'FD_none_Sexo_c18', 'FM_CardioMetab_Diabetes_c05', 'FS_Educ_yeduca', 
             'FS_Aislamiento_ViveSolo_g2',
                'FM_CardioMetab_Hiperten_c04', 'FM_CardioMetab_IAM_c08',
                 'FM_EstiloVida_Alcohol_c23', 'FM_EstiloVida_ActividadFis_c25a', 'FM_EstiloVida_Fuma_c24', 
                 'FM_EstiloVida_Caida12Mes_c11_med','FM_SaludMental_ProbNervDiagnost_c20', 'MMSE_diff', 'Barthel_diff']


list_drop.append('Barthel_w2')
list_drop.append('Barthel')
X_costarica_01 = X_costarica_01[list_drop]

In [ ]:
X_costarica_01.shape

In [ ]:
X_costarica_01_common_subjects = X_costarica_01.drop(['MMSE_diff' ], axis=1)
X_costarica_01_common_subjects.drop(X_costarica_01_common_subjects[X_costarica_01_common_subjects['Barthel_diff'] <0].index, inplace=True)
X_costarica_01_common_subjects = X_costarica_01_common_subjects.drop(['Barthel_diff' ], axis=1)
X_costarica_01_common_subjects.dropna(inplace=True)


print(X_costarica_01_common_subjects.shape,)

In [ ]:
X_costarica_w1 = X_costarica_01_common_subjects.drop(['Barthel_w2' ], axis=1)
X_costarica_w2 = X_costarica_01_common_subjects.drop(['Barthel' ], axis=1)

In [ ]:
print('w1:', X_costarica_w1.shape[0], '\w2:', X_costarica_w2.shape[0])
print('w1:', X_costarica_01.shape[0] - X_costarica_w1.shape[0], 
      '\tw2:', X_costarica_01_common_subjects.shape[0] - X_costarica_w2.shape[0])

In [ ]:
sabe_china_2011_2014_ = sabe_china_2011_2014[order_col_long]
sabe_china_2011_ = sabe_china_2011_2014[order_col_2011]
sabe_china_2014_ = sabe_china_2011_2014[order_col_2014]

sabe_china_2011_ = sabe_china_2011_.rename(columns={'Barthel_2011': 'Barthel'})
sabe_china_2014_ = sabe_china_2014_.rename(columns={'Barthel_2014': 'Barthel'})

In [ ]:
new_name = ['Age', 'Sex', 'Diabetes', 'Education', 'Hypertension', 'Heart Disease', 'Mental Problems', 'Physical activity', 'Alcohol consumption',
       'Social activity', 'Smoking status', 'Income', 'Barthel', 'Barthel_diff']

In [ ]:
sabe_china_2011_.columns =  new_name
sabe_china_2014_.columns =  new_name

In [ ]:
sabe_china_2011_['Barthel_diff'] = np.where(sabe_china_2011_['Barthel_diff'] < 0, np.nan, sabe_china_2011_['Barthel_diff'])
sabe_china_2011_.dropna(inplace=True)

sabe_china_2014_['Barthel_diff'] = np.where(sabe_china_2014_['Barthel_diff'] < 0, np.nan, sabe_china_2014_['Barthel_diff'])
sabe_china_2014_.dropna(inplace=True)

sabe_china_2011_ = sabe_china_2011_.drop(['Barthel_diff'], axis = 1)
sabe_china_2014_ = sabe_china_2014_.drop(['Barthel_diff'], axis = 1)


print(sabe_china_2011_.shape, sabe_china_2014_.shape)

In [ ]:
parameter_dict = {}

### Colors, variables names

In [ ]:
X_cat = pd.read_csv('../Data/cross/var_name_color_code_new.csv', encoding='latin-1', sep=";")
X_cat_china = pd.read_csv('../Data/cross/var_name_color_code_new_china.csv', encoding='latin-1', sep=";")

In [ ]:
new_name = []

for i in range(len(X_costarica_w1.columns)):
    if(X_costarica_w1.columns[i] == 'FM_EstiloVida_Caida12Mes_c11_med'):
        label = 'FM_EstiloVida_Caida12Mes_c11'
    else:
        label = X_costarica_w1.columns[i]
    
    index_ = list(X_cat.oldname).index(label)
    new_name.append(list(X_cat.newname)[index_])

In [ ]:
# Sanity Check
for i in range(len(X_costarica_w1.columns)):
    print('', X_costarica_w1.columns[i], '\n', X_costarica_w2.columns[i])
    print('-------------------------------------------')

In [ ]:
X_costarica_w1.columns =  new_name
X_costarica_w2.columns =  new_name


In [ ]:
# Sanity Check
for i in range(len(X_costarica_w1.columns)):
    print('', X_costarica_w1.columns[i], '\n', X_costarica_w2.columns[i])
    print('-------------------------------------------')

## Regression Models

### Costa Rica wave 1

In [ ]:
y_costarica_1 = X_costarica_w1['Barthel']
X_costarica_1 = X_costarica_w1.drop(['Barthel'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_costarica_1, y_costarica_1, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (1.0, 0.1, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')

In [ ]:
modelRidge_costarica_1 = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_costarica_1.fit(X_costarica_1, y_costarica_1)

parameter_dict['costarica_wave_1'] = opt_Ridge.best_params_

modelRidge_sum_costarica_1 = summary_validation( X_costarica_1, y_costarica_1, opt_Ridge)

### Costa Rica wave 2

In [ ]:
y_costarica_2 = X_costarica_w2['Barthel']
X_costarica_2 = X_costarica_w2.drop(['Barthel'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_costarica_2, y_costarica_2, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (1.0, 0.1, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')


In [ ]:
modelRidge_costarica_2 = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_costarica_2.fit(X_costarica_2, y_costarica_2)

parameter_dict['costarica_wave_2'] = opt_Ridge.best_params_

modelRidge_sum_costarica_2 = summary_validation(X_costarica_2, y_costarica_2, opt_Ridge)

### China 2011

In [ ]:
y_china_2011 = sabe_china_2011_['Barthel']
X_china_2011 = sabe_china_2011_.drop('Barthel', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_china_2011, y_china_2011, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (1.0, 0.1, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')

In [ ]:
modelRidge_china_2011 = Ridge(alpha=opt_Ridge.best_params_['alpha'], solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_china_2011.fit(X_china_2011, y_china_2011)

parameter_dict['china_2011'] = opt_Ridge.best_params_

modelRidge_sum_china_2011 = summary_validation(X_china_2011, y_china_2011, opt_Ridge)

### China 2014

In [ ]:
y_china_2014 = sabe_china_2014_['Barthel']
X_china_2014 = sabe_china_2014_.drop('Barthel', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_china_2014, y_china_2014, train_size=0.75, test_size=.25, random_state=0)

opt_Ridge = BayesSearchCV(
    Ridge(),
    {
        'alpha': (1.0, 0.1, 0.01, 0.001),
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'max_iter': (1000, 10000, 100000, 1000000),
        #'tol:' : (1e-6, 1e-3, 1e+1),
        #'n_estimators': (100, 1000),

    },
    n_iter=10,
    cv=3
)

opt_Ridge.fit(X_train, y_train)

print('Ridge')
print("best parameters: %s" % str(opt_Ridge.best_params_))
print('---------------------------------------------\n')

In [ ]:
modelRidge_china_2014 = Ridge(alpha=0.001, solver = opt_Ridge.best_params_['solver'], 
                         max_iter=opt_Ridge.best_params_['max_iter'])
modelRidge_china_2014.fit(X_china_2014, y_china_2014)

parameter_dict['china_2014'] = opt_Ridge.best_params_

modelRidge_sum_china_2014 = summary_validation(X_china_2014, y_china_2014, opt_Ridge)

## Significant variables

In [ ]:
col = ['Age', 'Sex', 'Diabetes', 'Education', 'Hypertension',
       'Heart Disease', 'Alcohol consumption', 'Physical activity',
       'Smoking status', 'Mental Problems']

df = modelRidge_sum_costarica_2

feature_sign = []

for i in col:
    if df[0].loc[i, 'p value stouffer'] < 0.05:
        feature_sign.append(i)


In [ ]:
feature_sign

In [ ]:
df_CR_c = pd.DataFrame(modelRidge_sum_costarica_2[1])
df_CH_c = pd.DataFrame(modelRidge_sum_china_2014[1])

df_CR_p = pd.DataFrame(modelRidge_sum_costarica_2[2])
df_CH_p = pd.DataFrame(modelRidge_sum_china_2014[2])


df_CR_t = pd.DataFrame(modelRidge_sum_costarica_2[3])
df_CH_t = pd.DataFrame(modelRidge_sum_china_2014[3])


df_CR_c['Countries'] = 'CR'
df_CH_c['Countries'] = 'CH'

df_CR_p['Countries'] = 'CR'
df_CH_p['Countries'] = 'CH'

df_CR_t['Countries'] = 'CR'
df_CH_t['Countries'] = 'CH'

df_c = pd.concat([df_CR_c, df_CH_c])
df_p = pd.concat([df_CR_p, df_CH_p])
df_t = pd.concat([df_CR_t, df_CH_t])

In [ ]:
df_c = df_c[feature_sign + ['Countries']]
df_p = df_p[feature_sign + ['Countries']]
df_t = df_t[feature_sign + ['Countries']]

In [ ]:
feature_sign

In [ ]:
from scipy.stats import shapiro
from scipy import stats

for feat in feature_sign:
    current_feat = df_t[feat]
    current_feat.dropna(inplace=True)
    stat, p = shapiro(current_feat)
    kurt = stats.kurtosis(current_feat)
    skew = stats.skew(current_feat)

    
    feat_name = feat.ljust(45)
    print('{} \t p-value: {} \tKursotis: {}  \tSkewness: {}'.format(feat_name, np.round(p,4), np.round(kurt,4), np.round(skew,4)))


## Mann-Whitney

In [ ]:
order = ['CR', 'CH']
plt.figure(figsize= (11,9))
plt.suptitle('Mann-Whitney estimate functionality');
#hue = "Countries"

my_colors = ["#222222",  "#999999"]

c = 1
for current_feat in feature_sign:
    plt.subplot(3,3,c)
   
    p_cr  = modelRidge_sum_costarica_2[0].loc[current_feat, 'p value stouffer']
    p_ch  = modelRidge_sum_china_2014[0].loc[current_feat, 'p value stouffer']
        
    df_c_ = df_c.copy()    
    
    comp = [("CR", "CH")]
    
    do_comp = True
    
        
    if(p_ch > 0.05):    
        df_c_[df_c_['Countries'] == 'CH']=-0

        do_comp = False
    
    ax = sns.violinplot(data=df_c_, x='Countries', y=current_feat, palette= my_colors)


    
    if(do_comp):
        test_results = add_stat_annotation(ax, data=df_c, x='Countries', y=current_feat, order=order,
                                            box_pairs=comp,
                                            test='Mann-Whitney', text_format='star',
                                            loc='inside', verbose=0)
    
    c+=1
    
    plt.xlim([-0.5, 1.5])
    plt.xticks([0,1], ['CR','CH'])
    plt.tight_layout(pad=3);
    plt.close()
    


## Statistically significant variables in  Costa Rica wave 2, Korea 2016 and China 2014

In [ ]:
features_for_test = ['Age', 'Sex', 'Diabetes', 'Education', 'Hypertension',
       'Heart Disease', 'Alcohol consumption', 'Physical activity',
       'Smoking status', 'Mental Problems']


df_CR = pd.DataFrame(modelRidge_sum_costarica_2[1])
df_CR = df_CR[features_for_test]
df_CR.columns = [f"CR_{col}" for col in df_CR.columns]




df_CH = pd.DataFrame(modelRidge_sum_china_2014[1])
df_CH = df_CH[features_for_test]
df_CH.columns = [f"CH_{col}" for col in df_CH.columns]



beta_df = pd.concat([df_CR, df_CH], axis=1)


result_stats_df = pd.DataFrame(
        index=['CH mean', 'CR mean', 'CR_CH test_p value'] ,
        columns=features_for_test
        )

from scipy import stats
import statsmodels.api

for i in features_for_test:


    
    result_stats_df.loc['CR_CH test_p value',i] = np.min([statsmodels.stats.weightstats.ttest_ind(list(beta_df.loc[:, 'CR_'+ i]), 
                                                                                          list(beta_df.loc[:, 'CH_'+ i]), 
                                                                                          alternative="two-sided",
                                                                                          usevar="unequal")[1]*3,1])
    



    result_stats_df.loc['CH mean',i] = beta_df.loc[:, 'CH_'+ i].mean()
    result_stats_df.loc['CR mean',i] = beta_df.loc[:, 'CR_'+ i].mean()
result_stats_df.T

## Summary Results

## Models

In [ ]:
modelRidge_sum_costarica_1[0]#.to_excel('Results/longitudinal/costarica_wave1_func.xlsx')
modelRidge_sum_costarica_2[0]#.to_excel('Results/longitudinal/costarica_wave2_func.xlsx')

modelRidge_sum_china_2011[0]#.to_excel('Results/longitudinal/china_wave1_func.xlsx')
modelRidge_sum_china_2014[0]#.to_excel('Results/longitudinal/china_wave2_func.xlsx')

## Models parameters

In [ ]:
parameter_pd = pd.DataFrame(parameter_dict).T
parameter_pd#.to_excel('Results/longitudinal/functionality_hyperparameters.xlsx')

## Stats

In [ ]:
result_stats_df.T#.to_excel('Results/longitudinal/stats_func.xlsx')